<h2>Segmenting and Clustering Neighborhoods in Toronto</h2>

<h3>Part 1: Preparing Data</h3>

In [1]:
#import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
#web-scraping
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(webpage.content, "html.parser")
# saving to DataFrame
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head(7)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


<strong>Pre-processing Data</strong>
<ol>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. We should split them to multiple rows.</li>
</ol>

In [3]:
# replace 'Not assigned to NaN'
df.replace('Not assigned', np.nan, inplace = True)
#item1: if df.Neighborhood is NaN and df.Borough is not NaN, then df.Neighborhood = df.Borough
df.Neighborhood.fillna(df.Borough, inplace = True)
#item2: drop rows with df.Neighborhood is NaN
df.dropna(inplace=True)
#item3: split combined Neighborhoods into multiple rows
df_toronto = \
(df.set_index(df.columns.drop('Neighborhood',1).tolist())
   .Neighborhood.str.split('/', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'Neighborhood'})
   .loc[:, df.columns]
)
df_toronto.head(7)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
5,M6A,North York,Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park


In [4]:
df_toronto.shape

(208, 3)